In [29]:
import time, os
from scipy.ndimage import maximum_filter1d, find_objects
import torch
import numpy as np
import tifffile
from tqdm import trange
from numba import njit, float32, int32, vectorize
import cv2
import fastremap
import gunpowder as gp
import math
import zarr
import imageio
import matplotlib.pyplot as plt

import logging
from cellpose import dynamics
# dynamics_logger = logging.getLogger(__name__)

#from cellpose import utils, metrics, transforms

import torch
from torch import optim, nn
# from torch import resnet_torch
TORCH_ENABLED = True 
torch_GPU = torch.device('cuda')
torch_CPU = torch.device('cpu')

ModuleNotFoundError: No module named 'matplotlib'

In [11]:
# This loads the zarr file
load_path = '/mnt/efs/shared_data/hack/data/20230811/20230811_raw.zarr'
f = zarr.open(load_path, 'r')
f['fov0/raw'].shape

(64, 640, 640)

In [19]:
def show_one_image(image_path):
    image = imageio.imread(image_path)
    plt.imshow(image)

In [26]:
# get a crop

mask = f['fov0/gt'][15:16, 150:350,150:350]
mask.shape

(1, 200, 200)

In [28]:
plt.imshow(mask)

NameError: name 'plt' is not defined

In [13]:
dynamics.masks_to_flows(f['fov0/gt'])

## USE THIS to take in the masks, compute the flows from them, and then save the flows

AttributeError: 'Array' object has no attribute 'max'

In [8]:
# Make an array key for the raw data and the ground truth
raw = gp.ArrayKey('RAW')
gt = gp.ArrayKey('GROUND_TRUTH')

#%%

random_location = gp.RandomLocation()
simple_augment = gp.SimpleAugment()
stack = gp.Stack(5)

elastic_augment = gp.ElasticAugment(
  control_point_spacing=(20, 20),
  jitter_sigma=(1.0, 1.0),
  rotation_interval=(0, math.pi/2))

normalize = gp.Normalize(raw)
intensity_augment = gp.IntensityAugment(
  raw,
  scale_min=0.8,
  scale_max=1.2,
  shift_min=-0.2,
  shift_max=0.2)
noise_augment = gp.NoiseAugment(raw)

pad_raw = gp.Pad(raw, None)
pad_gt = gp.Pad(gt, 0)



# Make this 'pipeline' thingy
source = tuple(gp.ZarrSource(
    store_path,
    {
      raw: f'fov{i}/raw',
      gt: f'fov{i}/gt'
    },
    {
      raw: gp.ArraySpec(interpolatable=True,
        voxel_size=(1,1)),
      gt: gp.ArraySpec(interpolatable=False,
        voxel_size=(1,1)),
    }) + normalize + pad_raw + pad_gt + random_location for i in [1])


# pipeline = source + random_location + simple_augment + elastic_augment + intensity_augment + noise_augment
pipeline = source
pipeline += gp.RandomProvider()
pipeline += simple_augment
pipeline += elastic_augment
pipeline += intensity_augment
pipeline += noise_augment
  # +stack
  # reject
  # )


# formulate a request for "raw"
request = gp.BatchRequest()
request[raw] = gp.Roi((0,0), (128, 128))
request[gt] = gp.Roi((0,0), (128, 128))

# build the pipeline...
with gp.build(pipeline):

    # ...and request a batch
    batch = pipeline.request_batch(request)

# show the content of the batch
print(f"batch returned: {batch}")

NameError: name 'store_path' is not defined